<a href="https://colab.research.google.com/github/Gobihanath/DTect/blob/main/DTect_Alter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import cv2

input_shape = (256,256 , 3)  # 3 means the rgb format

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Image_Width = 256
Image_Height = 256
Image_Size = (Image_Width, Image_Height)
Image_Channel = 3
batch_size=128

In [ ]:
data_set = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Dataset/Dtect | Dataset | New",
    seed=123,         #If you want to ensure that the shuffling of your dataset is the same across different runs, you can keep the seed value constant. If you don't care about reproducibility, you can omit the seed parameter or use a different value for each run.
    shuffle=True,
    image_size=Image_Size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 5001 files belonging to 10 classes.


In [ ]:
class_names = data_set.class_names
class_names

['Person 1',
 'Person 10',
 'Person 2',
 'Person 3',
 'Person 4',
 'Person 5',
 'Person 6',
 'Person 7',
 'Person 8',
 'Person 9']

In [ ]:
def get_dataset_partitions_tf(data_set, train_split=0.8,  val_split=0.2, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split ) == 1

    ds_size = len(data_set)

    if shuffle:
        data_set = data_set.shuffle(shuffle_size, seed=123)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = data_set.take(train_size)
    val_ds = data_set.skip(train_size).take(val_size)

    return train_ds, val_ds

In [ ]:
train_ds, val_ds = get_dataset_partitions_tf(data_set)

print(len(train_ds))
print(len(val_ds))

32
8


In [ ]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=input_shape))

model.add(layers.Conv2D(16, 3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D())



model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D())


model.add(layers.Conv2D(64, 3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))


model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10,activation='softmax'))


model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 64)       

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
ACCURACY_THRESHOLD = 0.85

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        accuracy = logs.get('accuracy')  # Use 'accuracy' instead of 'acc'
        if accuracy is not None and accuracy > ACCURACY_THRESHOLD:
            print("\nReached %2.2f%% accuracy, so stopping training!!" % (ACCURACY_THRESHOLD * 100))
            self.model.stop_training = True

# Instantiate a callback object
early_stopping = myCallback()

In [ ]:
history  = model.fit(
    train_ds,
    batch_size=128,
    validation_data=val_ds,
    validation_steps = 8,
    verbose=1,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
32/32 [==============================] - 728s 16s/step - loss: 2.3466 - accuracy: 0.1277 - val_loss: 2.1797 - val_accuracy: 0.2227
Epoch 2/20
32/32 [==============================] - 724s 16s/step - loss: 2.0324 - accuracy: 0.2942 - val_loss: 1.7285 - val_accuracy: 0.3926
Epoch 3/20
32/32 [==============================] - 710s 16s/step - loss: 1.5441 - accuracy: 0.4888 - val_loss: 1.3223 - val_accuracy: 0.5557
Epoch 4/20
32/32 [==============================] - 712s 16s/step - loss: 1.1567 - accuracy: 0.6047 - val_loss: 1.0172 - val_accuracy: 0.6453
Epoch 5/20
32/32 [==============================] - 721s 17s/step - loss: 0.7955 - accuracy: 0.7344 - val_loss: 0.6115 - val_accuracy: 0.7923
Epoch 6/20
32/32 [==============================] - 715s 16s/step - loss: 0.6107 - accuracy: 0.7885 - val_loss: 0.5647 - val_accuracy: 0.8066
Epoch 7/20
32/32 [==============================] - ETA: 0s - loss: 0.4328 - accuracy: 0.8542
Reached 85.00% accuracy, so stopping training!!
32/32 

In [ ]:
train_loss, train_acc = model.evaluate(train_ds)
print('Training accuracy:', train_acc)

val_loss, val_acc = model.evaluate(val_ds)
print('Validation accuracy:', val_acc)


32/32 [==============================] - 288s 2s/step - loss: 0.3141 - accuracy: 0.9075
Training accuracy: 0.9074679613113403
8/8 [==============================] - 240s 4s/step - loss: 0.3286 - accuracy: 0.8965
Validation accuracy: 0.896484375


# Checking the current learning rate

In [ ]:
optimizer = model.optimizer

# Get the learning rate from the optimizer's configuration
learning_rate = optimizer.get_config()['learning_rate']
print("Current learning rate:", learning_rate)

Current learning rate: 0.001


In [ ]:
model.save('/content/drive/MyDrive/ML Trained Models/Dtect_Alternation_Model_01.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
import tensorflow as tf

def predict(model, img, class_names, confidence_threshold=0.5):
    # Preprocess the image
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    # Make predictions
    predictions = model.predict(img_array)

    # Extract predicted class and confidence
    predicted_class_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_index]
    predicted_class = class_names[predicted_class_index]

    # Check if confidence is high enough
    if confidence >= confidence_threshold:
        verification_status = "verified"
    else:
        verification_status = "not verified"

    return predicted_class, confidence, verification_status

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/ML Trained Models/Dtect_Alternation_Model_01.h5')

# Define class names
# Add your class names here

# Load the image you want to predict
img = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/T3.jpg', target_size=(256, 256))

# Predict the image
predicted_class, confidence, verification_status = predict(model, img, class_names)

# Print the results
print("Predicted class:", predicted_class)
print("Confidence:", confidence)
print("Verification status:", verification_status)

1/1 [==============================] - 0s 178ms/step
Predicted class: Person7
Confidence: 0.6544799
Verification status: verified
